# Exercise 1 - Modifying the environment

We will make the problem slightly more complicated.

The floor is now covered in ice! 
When an agent makes a movement, it might slip and end up in another cell close to it.
##### The probability to arrive in the intended cell is 0.6, and the probability to end up in one of the 4 adjacent cells is 0.1.

Similar as for Lab 02, you should create a new Dungeon by inheriting from the original Dungeon environment.
##### Again, the step method will return the state instead of observations.
And the step function should incorportate these slippery dynamics.

##### When the agent slips, rewards accumulate! You could bang your head on the wall twice...


In [253]:
from dungeon.dungeon import Dungeon
import numpy as np
from collections import namedtuple, defaultdict

class IceDungeon(Dungeon):
    
    def __init__(self, N):
        
        super().__init__(N)
        
        # In order to explicitely show that the way you represent states doesn't matter, 
        # we will assign a random index for each coordinate of the grid        
        index_states = np.arange(0, N*N)
        np.random.shuffle(index_states)
        self.coord_to_index_state = index_states.reshape(N,N) #reshape!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        print(self.coord_to_index_state)
        
    def step(self, action):
        
        slip_actions = ('stay', 'up', 'down', 'left', 'right')
        obs, rew, done, bump = super().step(action)
        state = self.coord_to_index_state[ self.position_agent[0], self.position_agent[1]]
#         print(obs, rew, done, bump, state)
#         print(self.position_agent, 'time escape', self.time_elapsed)
        
        slip_action = np.random.choice(slip_actions, p=[0.6, 0.1, 0.1, 0.1, 0.1])
        
#         print(slip_action)
        if done == False and bump == False and slip_action != 'stay':
            obs, rew, done, bump = super().step(slip_action)
            
            state = self.coord_to_index_state[self.position_agent[0], self.position_agent[1]]
#             print(obs, rew, done, bump, state)
#             print(self.position_agent, 'time escape', self.time_elapsed)

        return obs, rew, done, bump, state
    
    def reset(self):
        
        super().reset()
        state = self.coord_to_index_state[ self.position_agent[0], self.position_agent[1]]
        #print(self.position_agent)
        
        return state
        

In [254]:
dungeon = IceDungeon(10)
dungeon.reset()
dungeon.display()

[[21 17 50 59 88 95 22 11 52 75]
 [65 97 44 47 23 16 85 86 25 40]
 [41  9 91 84 31 35 56 73 70 90]
 [87 30 38  4 14 28 37 32 53 74]
 [43 67 39 78 63 98 89  6 99  8]
 [61 42 76 93 26 15  0 92 10 51]
 [68  3 18  7 71 58  2 34 24 96]
 [77  1 80 94 29 49 48 27 54 19]
 [69 64 79 83 62 66 81 45 12 36]
 [46 33 72 13 20 57 60  5 55 82]]
X X X X X X X X X X 
X . . . X . . . . X 
X X . X . . X . . X 
X L . . . L . . . X 
X . . . . . . . L X 
X . . A . . . . . X 
X L . . . . . . . X 
X . . . . . . L . X 
X . . . . . X E . X 
X X X X X X X X X X 



In [255]:
a = dungeon.step('up')

# Exercise 2 - Monte-carlo learning

We will use a simple random policy function to evaluate the state values. 

First of all, create a random policy function that just picks random actions (check Lab 01).

Then, implement a MC learning class that allows to learn the values based on full rollouts of the policy in the environment.

Finally, you can generate rollouts of your policy in an environment, and update the values using MC-learning.

In [278]:
def random_policy():
    
    return np.random.choice(['up', 'down', 'left', 'right'])
    
class MC_Learning():
    
    def __init__(self, N, environment, policy, gamma, episodes):
        self.N = N
        self.environment = environment
        self.policy = policy
        self.gamma = gamma
        self.episodes = episodes
        self.values = {}
    
#     def run_single_exp(envir, policy):
    
#         obs = envir.reset()
#         done = False
    
#         total_reward = 0
    
#         while not done:
#             action = policy(obs)
#             obs, reward, done = envir.step(action)
#             total_reward += reward
        
#         return total_reward
    
#     def generate_episodes(self):
#         self.policy
#         G = 0
#         states_and_returns = []
#         first = True
#         for s, r in reversed(states_and_rewards):
#             print('11111111',s, r)
#             # the value of the terminal state is 0 by definition
#             # we should ignore the first state we encounter
#             # and ignore the last G, which is meaningless since it doesn't correspond to any move
#         if first:
#             first = False
#         else:
#             states_and_returns.append((s, G))
#         G = r + GAMMA*G
#         states_and_returns.reverse() # we want it to be in order of state visited
#         return states_and_returns
    
    def generate_episode(self): #, rollout
        
        state = self.environment.reset()
#         self.environment.display()#####################################################
        
        states_and_rewards = [(state, 0)]
        done = False
        total_reward = 0
        
        while not done:
            action = self.policy()
            obs, rew, done, bump, state = self.environment.step(action)
            #print(action,rew,state)
            states_and_rewards.append((state,rew))
            #total_reward += reward     
        #print(states_and_rewards)
        #print(done)
        
        G = 0
        states_and_returns = []
        first = True
        for state, rew in reversed(states_and_rewards):
            if first:
                first = False
            else:
                states_and_returns.append((state,G))
            G = rew + self.gamma * G
        states_and_returns.reverse()
        #print(states_and_returns)
        
        return states_and_returns
    
    def update_values(self): #, rollout
        
        # print(self.values)
        returns = {}
        states = self.environment.coord_to_index_state.reshape(self.N*self.N)
        
        for s in states:
            returns[s] = []
        print(returns)
            
        for episode in range(self.episodes):
            states_and_returns = self.generate_episode()
            seen_states = set()
            #print(states_and_returns)
            for state, G in states_and_returns:
                #print(state,G)
                if state not in seen_states:
                    returns[state].append(G)
                    self.values[state] = np.mean(returns[state])
                    seen_states.add(state)
        print(self.values)
            
    def display_values(self):
        ...

In [ ]:
dungeon = IceDungeon(10)
dungeon.reset()
dungeon.display()
test = MC_Learning(10, dungeon, random_policy, 0.9, 9999)
test.update_values()

[[29 22 91 94 64 37 77 70 76 52]
 [87 83 42 81 26 43 11 75 85  3]
 [69 21 20 58  5 59 45  1 39 12]
 [17 49 33 80 84 10 16 55 97 34]
 [89 78 71 62 92 13 19 36 51  6]
 [15  0 61 98 35 73 88  8 30 72]
 [63 74 53 14 46 86 48 56 31 68]
 [40 47 79  9 28 54 93 96  7 99]
 [90 50 24 67 82  4 65 95 25 66]
 [41 44 60  2 18 27 23 38 32 57]]
X X X X X X X X X X 
X . . . . . L . . X 
X . . L . . . . . X 
X . . . . . . . . X 
X . . . . . . . . X 
X L . . X A . E . X 
X . . . X . . X . X 
X L . . . X . . . X 
X . L . . . . X . X 
X X X X X X X X X X 

{29: [], 22: [], 91: [], 94: [], 64: [], 37: [], 77: [], 70: [], 76: [], 52: [], 87: [], 83: [], 42: [], 81: [], 26: [], 43: [], 11: [], 75: [], 85: [], 3: [], 69: [], 21: [], 20: [], 58: [], 5: [], 59: [], 45: [], 1: [], 39: [], 12: [], 17: [], 49: [], 33: [], 80: [], 84: [], 10: [], 16: [], 55: [], 97: [], 34: [], 89: [], 78: [], 71: [], 62: [], 92: [], 13: [], 19: [], 36: [], 51: [], 6: [], 15: [], 0: [], 61: [], 98: [], 35: [], 73: [], 88: [], 8: [],

In [ ]:
dungeon.display()

# Exercise 3: TD-learning

We will now compare the values obtained by MC learning with values obtained with TD learning.

Create a class that allows to update values every time a new state transition occurs.


In [ ]:
class TD_Learning():
    
    def __init__(self, N):
        
        self.values = ...
    
    def update_values(self, s_current, reward_next, s_next):
        ...
        
    def display_values(self):
        ...
        